# 2. Neural Style Transfer on your local machine

To make sure all the scripts that are working correctly, we need to test them on our local machine first. Specifically, we'll be testing the components that will be in our __flask app__ and our __scoring app__.

### Flask App
The __flask app__ is responsible for:
1. preprocessing the images into individual frames and extracting the audio file
2. adding references of the individual images to the Service Bus queue
3. waiting until style transfer has been applied to all images
4. postprocessing the style transfer images by stitching them together into a video and re-adding the audio clip

The __flask app__ contains the following files:
- `preprocess.py` 
- `add_images_to_queue.py` 
- `postprocess.py` 
- `util.py`
- `main.py`

`preprocess.py`, `add_images_to_queue.py`, and `postprocess.py` can be executed independently. This is to allow us to test each piece. The `main.py` file will call functions the aforementioned files. It will start by preprocessing the video then adding images to the service bus queue. Then it will wait for the messages in the queue to be dequeued and processed by another running process. (This occurs in the __scoring app__.) After all the images in the queue are processed, the `main.py` will start to postprocess the output frames, and recreate a new video, but this time with the style transfered frames.

### Scoring App
The __scoring app__ is responsible for:
1. polling the Service Bus queue for new images 
2. applying style transfer to those images and saving them back to storage

The __scoring app__ containers the following files:
- `main.py`
- `process_images_from_queue.py`
- `style_transfer.py`
- `util.py`

`main.py` starts the app by calling on the function in the `process_images_from_queue.py` file. This is responsible for continuously polling the service bus queue and checking for new messages. As images get pulled off the queue, the process will then call the stylize function in the `style_transfer.py` file which performs the actual style transfer. The generated style images are saved back to blob directly from this process. 

---

### Import package and load .env

In [1]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import os

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

### Preprocess video 
Run the preprocessing step on the video `orangutan.mp4` that is in blob. Specify the video (including the extention) and the mounted directory to access blob. 

In [3]:
!python flask_app/preprocess.py \
    --video orangutan.mp4 \
    --storage-mount-dir data

ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/jiata/anaconda3/envs/batchscoringdl --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilter     7. 16.100 /  7. 16.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  1.100 /  5.  1.100
  libswresample   3.  1.100 /  3.  1.100
  libpostproc    55.  1.100 / 55.  1.100
Input #0, mov,mp4,m4a

### Add messages to queue
The preprocessing step splits the video into individual frames. Now add those frames to the queue to be processed later.

In [4]:
!python flask_app/add_images_to_queue.py \
    --video-name orangutan \
    --storage-mount-dir data

### Run the main style transfer process
The `scoring_app/main.py` file will dequeue images from the queue and apply style transfer to the images.

In [5]:
!python scoring_app/main.py --terminate

### Postprocess the frames
Now that the frames have been processed, we can stitch then back together and reattach the audio file using the `flask_app/postprocess.py` file.

In [8]:
!python flask_app/postprocess.py \
    --video-name orangutan \
    --storage-mount-dir data

ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/jiata/anaconda3/envs/batchscoringdl --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilter     7. 16.100 /  7. 16.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  1.100 /  5.  1.100
  libswresample   3.  1.100 /  3.  1.100
  libpostproc    55.  1.100 / 55.  1.100
Input #0, image2, fro

Play the video! It will be called `{video_name}_processed.mp4`

In [10]:
%%HTML
<video width="320" height="240" controls>
  <source src="data/orangutan/orangutan_processed.mp4" type="video/mp4">
</video>

### Additional commands to monitor
Use these additional commands to monitor.

```bash
# checkout how many elements are in the queue
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query "countDetails.activeMessageCount"
```

---

Continue to the next [notebook](/notebooks/03_create_aks_cluster.ipynb).